In [43]:
# Install dependencies with pip (needed for Codespaces)
%pip install numpy
%pip install qiskit
%pip install qiskit_ibm_runtime
%pip install matplotlib

import sys
import math
import base64
import random
import hashlib
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Operator
from numpy import pi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [44]:
import sys
import random

# Increase the recursion limit for deep recursive functions
sys.setrecursionlimit(1500)

def gcd(a, b):
    """Compute the greatest common divisor of a and b."""
    while b:
        a, b = b, a % b
    return a

def modular_pow(base, exponent, modulus):
    """Perform modular exponentiation."""
    result = 1
    base = base % modulus
    while exponent > 0:
        if (exponent % 2) == 1:  # If exponent is odd, multiply base with the result
            result = (result * base) % modulus
        exponent = exponent >> 1  # exponent = exponent / 2
        base = (base * base) % modulus  # base = base * base
    return result

def fermat(p, k=2):
    """Test if p is a prime number using Fermat's little theorem, repeated k times."""
    if p < 2:
        return False
    if p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]:
        return True
    if any(p % i == 0 for i in [2, 3, 5]):
        return False
    for _ in range(k):
        a = random.randint(2, p - 2)
        if modular_pow(a, p - 1, p) != 1:
            return False
    return True

def karatsuba(x, y):
    """Multiply x and y using the Karatsuba multiplication algorithm."""
    if x < 10 or y < 10:
        return x * y
    m = max(len(str(x)), len(str(y))) // 2
    high1, low1 = divmod(x, 10**m)
    high2, low2 = divmod(y, 10**m)
    z0 = karatsuba(low1, low2)
    z1 = karatsuba((low1 + high1), (low2 + high2))
    z2 = karatsuba(high1, high2)
    return z2 * 10**(2*m) + ((z1 - z2 - z0) * 10**m) + z0

def mod_inverse(a, m):
    """Compute the modular inverse of a under modulo m using Extended Euclidean Algorithm."""
    m0, x0, x1 = m, 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def generate_e(phi):
    """Find an appropriate e and compute d using mod inverse."""
    for e in [65537, 32767, 17011, 8191, 4093, 2039, 1021, 509, 257, 127, 61, 37, 17]:
        if gcd(e, phi) == 1:
            d = mod_inverse(e, phi)
            return {'e': e, 'd': d}
    return None

def generate_rsa_keys():
    """Generate RSA public and private keys."""
    primes = []
    while len(primes) < 2:
        p = random.randint(2**15, 2**16)
        if fermat(p):
            primes.append(p)
    p, q = primes
    n = karatsuba(p, q)
    phi = karatsuba(p-1, q-1)
    e_d = generate_e(phi)
    return {'p': p, 'q': q, 'n': n, 'phi': phi, 'e': e_d['e'], 'd': e_d['d']}

keys = generate_rsa_keys()
print("Generated RSA keys:", keys)


Generated RSA keys: {'p': 39103, 'q': 50893, 'n': 1990068979, 'phi': 1989978984, 'e': 65537, 'd': 1792247273}


In [48]:
%pip install pylatexenc

# Necessary imports
import os
import random
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from math import pi, log, ceil, sqrt, isqrt
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

def classical_preprocessing(N):
    print(f"Starting preprocessing for N: {N}")
    if N % 2 == 0:
        return f"{N} is even, no need for quantum algorithm."

    # Calculate the integer square root of N which is the maximum base to check
    max_base = isqrt(N)
    print(f"Max base to check: {max_base}")

    # Check if N is a perfect power: N = b^p for some integer b and p >= 2
    for base in range(2, max_base + 1):
        # Start with the square of the base and increase power
        current_power_value = base * base
        power = 2
        while current_power_value <= N:
            if current_power_value == N:
                return f"{N} is a power of {base}^{power}, no need for quantum algorithm."
            current_power_value *= base
            power += 1

    print("Completed preprocessing without finding power factors.")
    return None

def c_amod15(a, power, n):
    total_qubits = n + 1
    U = QuantumCircuit(total_qubits)

    for _ in range(power):
        for q in range(n):
            U.swap(q, (q + 1) % n)
        U.x(range(n))
        U.append(QuantumCircuit(n).to_gate(), range(n))

    return U.to_gate().control(1)

def qft_dagger(n):
    qc = QuantumCircuit(n)
    for j in range(n):
        for k in range(j):
            qc.cp(-pi/float(2**(j-k)), k, j)
        qc.h(j)
    return qc

def order_finding_circuit(N, a):
    n = ceil(log(N, 2))
    total_qubits = 2 * n + 1
    print(f"Number of qubits required: {total_qubits}")

    qc = QuantumCircuit(total_qubits, n)
    qc.h(range(n))
    print("Hadamard gates applied.")

    for q in range(n):
        qc.append(c_amod15(a, 2**q, n), [q] + list(range(n, 2*n)) + [2*n])
        print(f"Applied controlled modular exponentiation gate for q = {q}.")
    
    qc.append(qft_dagger(n), range(n))
    print("Quantum Fourier Transform applied.")

    qc.measure(range(n), range(n))
    print("Measurement applied.")

    print("Quantum circuit constructed successfully.")
    return qc


def modular_pow(base, exponent, modulus):
    """Efficient modular exponentiation function"""
    result = 1
    base = base % modulus  # Update base if it's more than the modulus
    while exponent > 0:
        # If exponent is odd, multiply the base with the result
        if (exponent % 2) == 1:
            result = (result * base) % modulus
        # Right shift the exponent by 1 and square the base
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def miller_rabin(n, k=10):
    """ Perform the Miller Rabin primality test on a given number n, k times """
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False

    # Write n-1 as d*2^s by factoring powers of 2 from n-1
    s = 0
    d = n - 1
    while d % 2 == 0:
        d >>= 1
        s += 1

    def trial_composite(a):
        x = modular_pow(a, d, n)
        if x == 1 or x == n - 1:
            return False
        for i in range(s - 1):
            x = modular_pow(x, 2, n)
            if x == n - 1:
                return False
        return True

    for _ in range(k):  # number of trials
        a = random.randrange(2, n - 1)
        if trial_composite(a):
            return False

    return True


IBM_TOKEN = os.getenv('IBM_QUANTUM_TOKEN')
if IBM_TOKEN:
    QiskitRuntimeService.save_account(
        channel="ibm_quantum",
        token=IBM_TOKEN,
        instance="ibm-q/open/main",
        set_as_default=True,
        overwrite=True
    )
    print("IBM Quantum account successfully configured.")
else:
    print("IBM Quantum Token not set in environment variables.")

service = QiskitRuntimeService()
backend = service.backend("ibmq_qasm_simulator")
print("Backend loaded:")

# Generate RSA keys
keys = generate_rsa_keys()  # Ensure your RSA key generation is compatible
print("Generated RSA keys:", keys)

# Check if the number is prime
if miller_rabin(keys['n']):
    print(f"{keys['n']} is prime, no need for quantum algorithm.")
else:
    print(f"{keys['n']} is not prime, proceeding with Shor's algorithm.")
    N = keys['n']
    a = 7  # You can choose a different base if needed

    preprocessing_result = classical_preprocessing(N)
    qc = order_finding_circuit(N, a)
    print("Starting transpilation...")
    transpiled_circuit = transpile(qc, backend, optimization_level=3)
    print("Transpilation complete.")

    # Optimize the circuit
    optimized_circuit = transpiled_circuit
    print("Circuit optimization complete.")

    # Reduce the number of shots
    shots = 1024  # Set the number of shots
    print(f"Reducing the number of shots to: {shots}")

    print("Submitting the job to the backend...")
    sampler = Sampler(backend)
    job = sampler.run(optimized_circuit, shots=shots)
    print(f"Job ID: {job.job_id()}")

    print("Waiting for job to complete...")
    result = job.result()
    print("Job completed. Processing results...")

    print("RESULTS:", result)
    quasi_dists = result.quasi_dists[0]  # Taking the first result if multiple

    print("Generating plot...")
    plt.bar(quasi_dists.keys(), quasi_dists.values())
    plt.xlabel('Measured State')
    plt.ylabel('Probability')
    plt.title('Measurement Outcomes from Shors Algorithm')
    plt.xticks(rotation=70)
    plt.show()
    print("Plot displayed.")


Note: you may need to restart the kernel to use updated packages.
IBM Quantum account successfully configured.
Backend loaded:
Generated RSA keys: {'p': 60029, 'q': 48673, 'n': 2921791517, 'phi': 2921682816, 'e': 65537, 'd': 1013585921}
2921791517 is not prime, proceeding with Shor's algorithm.
Starting preprocessing for N: 2921791517
Max base to check: 54053
Completed preprocessing without finding power factors.
Number of qubits required: 65
Hadamard gates applied.
Applied controlled modular exponentiation gate for q = 0.
Applied controlled modular exponentiation gate for q = 1.
Applied controlled modular exponentiation gate for q = 2.
Applied controlled modular exponentiation gate for q = 3.
Applied controlled modular exponentiation gate for q = 4.
Applied controlled modular exponentiation gate for q = 5.
Applied controlled modular exponentiation gate for q = 6.
Applied controlled modular exponentiation gate for q = 7.
Applied controlled modular exponentiation gate for q = 8.
Applied

In [ ]:
from math import ceil, log

N = 2234520179
n = ceil(log(N, 2))
print("Value of n:", n)

Value of n: 32
